<div style="position: relative;">
<img src="https://user-images.githubusercontent.com/7065401/98728503-5ab82f80-2378-11eb-9c79-adeb308fc647.png"></img>

<h1 style="color: white; position: absolute; top:27%; left:10%;">
    MySQL and MariaDB for Python Developers
</h1>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:55%; left:10%;">
    David Mertz, Ph.D.
</h3>

<h3 style="color: #ef7d22; font-weight: normal; position: absolute; top:62%; left:10%;">
    Data Scientist
</h3>
</div>

# MySQL Administration Tools

In concept, you could do all of your MySQL administration via Python commands through an adapter. Everything that can be configured can be configured via SQL commands.  Most likely, however, you would find this unecessarily cumbersome; using native administration tools is easier for most tasks.

Administration tools may be either command-line—i.e. the `mysql` administration tool—or GUI tools.  Although the tool MySQL Workbench (https://www.mysql.com/products/workbench/) is developed by Oracle, who holds the MySQL copyright, it is not the dominant tool in the way the the roughly corresponding tool `pgAdmin` is for PostgreSQL.  However, MySQL Workbench is flexible and free..

A number of commercial tools exist that either focus on MySQL specifically, or generically interact with many RDBMS servers.  In the Free Software world, phpMyAdmin (https://www.phpmyadmin.net/) is quite capable.  Although, as the name suggests, the tool is written in PHP and exposed via a web interface, you do not need to know or use PHP code to use it.  HeidiSQL (https://www.heidisql.com/) is a similar tool, as in OmniDB (https://omnidb.org/).

Each of the GUI tools have their own advantages and disadvantages, but in general they all look fairly similar and have largely the same capabilities. The one we will present in this lesson is DBeaver (https://dbeaver.io/).  Of all the tools, this is probably compatible with the largest number of database systems, both relational and NoSQL style ones.

---

We will talk a bit about DBeaver in this lesson, but the greater part will be spent on mysql because command-line tools enable better reproducibility and can be scripted more easily.

## DBeaver

Rather than walk through numerous screenshots, I will perform a few simple tasks within the DBeaver interface.

* Create a new connection to MySQL `ine` database
* Select the `ine` database
* Unfold the tree to identify the Tables within it
* View the table `census_zipcode_geography`
* Look at Properties tab (schema)
* Look at Data tab
* Filter for AWATER_SQMI > 200
* Sort by zipcode using column header icon
* Show adding selection using column header icon
* Use the SQL Editor to run:
```sql
SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                     FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                  FROM census_zipcode_geography)
ORDER BY awater_sqmi + aland_sqmi
LIMIT 1000;
```
* Present the graphical explanation of the query plan using side tab
* Look at the statistics for the query plan

## mysql command line

As with the GUI tool, a walk through of using `mysql` will make its usage much more clear than simply capturing screenshots or text portions of the interface.

**Configure the command-line tools**

```bash
mysql_config_editor set --login-path=client --host=localhost --user=ine_student --password
mysql_config_editor print --all
mysqldump ine users
mysql_config_editor set --login-path=mysqldump --host=localhost --user=ine_student --password
mysql_config_editor print --all
cat ~/.mylogin.cnf
```

**Working with tools**

```bash
mysqldump ine users
mysql ine
mysql> help
mysql> help contents
mysql> help datatypes
mysql> help bigint
mysql> SHOW DATABASES;
mysql> SHOW TABLES;
mysql> DESCRIBE Tweets;
mysql> SELECT * FROM Tweets LIMIT 2;
mysql> SELECT * FROM Tweets LIMIT 2\G

```

**Perform the nested query in the GUI example**

```sql
SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                   FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                FROM census_zipcode_geography)
ORDER BY awater_sqmi + aland_sqmi
LIMIT 1000;

EXPLAIN SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                   FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                FROM census_zipcode_geography)
ORDER BY awater_sqmi + aland_sqmi
LIMIT 1000\G

EXPLAIN ANALYZE SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                   FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                FROM census_zipcode_geography)
ORDER BY awater_sqmi + aland_sqmi
LIMIT 1000\G
```

**Create an index on areas**

```sql
CREATE INDEX water_sqmi ON census_zipcode_geography (awater_sqmi);
CREATE INDEX land_sqmi ON census_zipcode_geography (aland_sqmi);
SHOW INDEXES FROM census_zipcode_geography\G

EXPLAIN ANALYZE SELECT usps  
FROM census_zipcode_geography 
WHERE awater_sqmi > (SELECT avg(aland_sqmi) 
                   FROM census_zipcode_geography)
AND aland_sqmi > (SELECT avg(awater_sqmi)
                FROM census_zipcode_geography)
ORDER BY awater_sqmi + aland_sqmi
LIMIT 1000\G
```

**Client as tool in pipes**

Using `mysql` is perhaps most powerful when you can use it as a scripting engine to pass along SQL from a file or from STDIN. 

We might run a query from the shell (perhaps piping or redirecting the response).  By default, `mysql` produces tab separated results, which are easy to process with other tools.

In [1]:
!echo "SELECT * FROM users;" | mysql ine

user_id	username	password	age	created_on
1	Alice	bad_pw_1	37	2021-01-09 22:19:03
2	Bob	bad_pw_2	NULL	2021-01-09 22:19:03
3	Carlos	bad_pw_3	62	2021-01-09 22:19:03
10	Sybil	M7c&sd31&0hA	44	2021-01-09 22:32:16
11	Trudy	y9bD6SA2O%$t	22	2021-01-09 22:32:16
12	Vanna	9$Ts9HK*3!tR	55	2021-01-09 22:32:16


If you would like more formatted results, the switches `--html` and `--table` are availabe.

In [2]:
!echo "SELECT * FROM users;" | mysql --table ine

+---------+----------+--------------+------+---------------------+
| user_id | username | password     | age  | created_on          |
+---------+----------+--------------+------+---------------------+
|       1 | Alice    | bad_pw_1     |   37 | 2021-01-09 22:19:03 |
|       2 | Bob      | bad_pw_2     | NULL | 2021-01-09 22:19:03 |
|       3 | Carlos   | bad_pw_3     |   62 | 2021-01-09 22:19:03 |
|      10 | Sybil    | M7c&sd31&0hA |   44 | 2021-01-09 22:32:16 |
|      11 | Trudy    | y9bD6SA2O%$t |   22 | 2021-01-09 22:32:16 |
|      12 | Vanna    | 9$Ts9HK*3!tR |   55 | 2021-01-09 22:32:16 |
+---------+----------+--------------+------+---------------------+
